In [1]:
import numpy as np
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, ErrorCode, Variable, ParcelsRandom, DiffusionUniformKh, AdvectionDiffusionM1, AdvectionDiffusionEM
from datetime import timedelta
from glob import glob
import math
import xarray as xr

In [2]:
# Particle set

class OilParticle(ScipyParticle):
    U10 = Variable('U10', dtype=np.float32, initial = 0)
    V10 = Variable('V10', dtype=np.float32, initial = 0)
    wind_speed = Variable('wind_speed', dtype=np.float32, initial = 0)

In [3]:
# Add MOI field set

data_path = '/storage/shared/oceanparcels/input_data/MOi/'
ufiles = sorted(glob(data_path+'psy4v3r1/psy4v3r1-daily_U_2019-06-*.nc'))
vfiles = [f.replace('_U_', '_V_') for f in ufiles]
wfiles = [f.replace('_U_', '_W_') for f in ufiles]
mesh_mask = data_path + 'domain_ORCA0083-N006/coordinates.nc'

filenames = {'U': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': ufiles},
             'V': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': vfiles},
             'W': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': wfiles}}
variables = {'U': 'vozocrtx', 'V': 'vomecrty', 'W': 'vovecrtz'}
dimensions = {'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}}

fieldset_moi = FieldSet.from_nemo(filenames, variables, dimensions)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [4]:
# Add wind fieldset from netcdf file

wind_dataset = xr.open_dataset('era_5_wind_data_june_2019.nc')

filenames_t_wind = {'U': 'era_5_wind_data_june_2019.nc', 'V': 'era_5_wind_data_june_2019.nc'}
variables_t_wind = {'U': 'u10', 'V': 'v10'}
dimensions_t_wind = {'U': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'},
              'V': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'}}

fieldset_wind = FieldSet.from_netcdf(filenames_t_wind, variables_t_wind, dimensions_t_wind)

In [5]:
# Add wind speed fields to fieldset


u_wind_total = Field('U10', wind_dataset['u10'], grid=fieldset_wind.U.grid)
v_wind_total = Field('V10', wind_dataset['v10'], grid=fieldset_wind.V.grid)

fieldset_moi.add_field(u_wind_total)
fieldset_moi.add_field(v_wind_total)

fieldset_moi.add_constant('wind_factor', 0.02)

In [6]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [7]:
def OilAdvectionRK4(particle, fieldset, time):
    
    if particle.depth == 0:
        (u1, v1) = fieldset.UV[particle]
        u10_1 = fieldset.U10[particle] * fieldset.wind_factor
        v10_1 = fieldset.V10[particle] * fieldset.wind_factor
        lon1, lat1 = (particle.lon + (u1 + u10_1)*.5*particle.dt, particle.lat + (v1 + v10_1)*.5*particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        u10_2 = fieldset.U10[time + .5 * particle.dt, particle.depth, lat1, lon1, particle] * fieldset.wind_factor
        v10_2 = fieldset.V10[time + .5 * particle.dt, particle.depth, lat1, lon1, particle] * fieldset.wind_factor
        lon2, lat2 = (particle.lon + (u2 + u10_2)*.5*particle.dt, particle.lat + (v2 + v10_2)*.5*particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        u10_3 = fieldset.U10[time + .5 * particle.dt, particle.depth, lat2, lon2, particle] * fieldset.wind_factor
        v10_3 = fieldset.V10[time + .5 * particle.dt, particle.depth, lat2, lon2, particle] * fieldset.wind_factor
        lon3, lat3 = (particle.lon + (u3 + u10_3)*particle.dt, particle.lat + (v3 + v10_3)*particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3, particle]
        u10_4 = fieldset.U10[time + particle.dt, particle.depth, lat3, lon3, particle] * fieldset.wind_factor
        v10_4 = fieldset.V10[time + particle.dt, particle.depth, lat3, lon3, particle] * fieldset.wind_factor
        particle.lon += ((u1 + u10_1) + 2*(u2 + u10_2) + 2*(u3 + u10_3) + (u4 + u10_4)) / 6. * particle.dt
        particle.lat += ((v1 + v10_1) + 2*(v2 + v10_2) + 2*(v3 + v10_3) + (v4 + v10_4)) / 6. * particle.dt
        
    else:
        (u1, v1) = fieldset.UV[particle]
        lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3, particle]
        particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
        particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt

In [8]:
def OilAdvectionEE(particle, fieldset, time):
    """Advection of particles using Explicit Euler (aka Euler Forward) integration.
    Function needs to be converted to Kernel object before execution"""
    u1 = fieldset.U[particle]
    v1 = fieldset.V[particle]
    u10 = fieldset.U10[particle] * fieldset.wind_factor
    v10 = fieldset.V10[particle] * fieldset.wind_factor
    particle.lon += (u1 + u10) * particle.dt
    particle.lat += (v1 + v10) * particle.dt

In [9]:
# Loading the wind speed data test, needed later for different kernels

def LoadingWindSpeedTest(particle, fieldset, time):
    
    particle.U10 = fieldset.U10[particle]
    particle.V10 = fieldset.V10[particle]

In [10]:
number_particles = 5

lonp = 2 * np.ones(number_particles)
latp = 60 * np.ones(number_particles)
depp = np.zeros(number_particles)

In [11]:
# Load particle set with the MOI fieldset

pset = ParticleSet(fieldset=fieldset_moi, pclass=OilParticle, lon=lonp, lat=latp, depth=depp)

In [12]:
kernels = pset.Kernel(OilAdvectionRK4) + pset.Kernel(LoadingWindSpeedTest)

output_file = pset.ParticleFile(name="test_wind_speed_advection_kernel", outputdt=timedelta(minutes=10))

pset.execute(kernels, runtime=timedelta(minutes=30), dt=timedelta(seconds=30), recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle}, output_file=output_file)

output_file.close()

INFO: Temporary output files are stored in out-FRMZPKFM.
INFO: You can use "parcels_convert_npydir_to_netcdf out-FRMZPKFM" to convert these to a NetCDF file during the run.
N/A% (0 of 1800.0) |                     | Elapsed Time: 0:00:00 ETA:  --:--:--WARNING: ParticleSet is empty on writing as array at time 1200
 66% (1200.0 of 1800.0) |##########      | Elapsed Time: 0:00:10 ETA:   0:00:05WARNING: ParticleSet is empty on writing as array at time 1800
100% (1800.0 of 1800.0) |################| Elapsed Time: 0:00:18 Time:  0:00:18


In [13]:
ds = xr.open_dataset('test_wind_speed_advection_kernel.nc')

ds

<xarray.Dataset>
Dimensions:     (traj: 5, obs: 2)
Dimensions without coordinates: traj, obs
Data variables:
    trajectory  (traj, obs) float64 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float64 ...
    lon         (traj, obs) float64 ...
    z           (traj, obs) float64 ...
    U10         (traj, obs) float32 ...
    V10         (traj, obs) float32 ...
    wind_speed  (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        v2.3.0-81-g6696f217
    parcels_mesh:           spherical